## Data Harvesting

## Retrieve Samenmeten sensor locations 

In [1]:
import requests

# List of LTD coulumn values
ltd_values = ['LTD_4969', 'LTD_32517', 'LTD_23231', 'LTD_8802', 'LTD_32999', 'LTD_55037']

headers = {"Content-Type": "application/json"}

def get_coordinates(thing):
    locations_link = thing.get('Locations@iot.navigationLink')

    if locations_link:
        location_response = requests.get(locations_link, headers=headers)
        if location_response.status_code == 200:
            location_data = location_response.json()
            coordinates = location_data.get('value', [{}])[0].get('location', {}).get('coordinates', [])
            lng, lat = coordinates if len(coordinates) == 2 else (None, None)
        else:
            lng, lat = None, None
    else:
        lng, lat = None, None

    return lng, lat

for ltd_value in ltd_values:
    url = f"https://api-samenmeten.rivm.nl/v1.0/Things?$filter=name eq '{ltd_value}'"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        # Check if the 'value' key is present in the response
        if 'value' in data:
            things = data['value']
            for thing in things:
                thing_id = thing.get('@iot.id')
                name = thing.get('name')
                description = thing.get('description')
                lng, lat = get_coordinates(thing)

                # Print the desired information
                print(f"@iot.id: {thing_id}, name: {name}, description: {description}, location: ({lng}, {lat})")

        else:
            print(f"No 'value' key in the response for {ltd_value}.")

    else:
        print(f"Error for {ltd_value}: {response.status_code}")


@iot.id: 921, name: LTD_4969, description: LTD_4969, location: (4.968, 52.304)
@iot.id: 2777, name: LTD_32517, description: LTD_32517, location: (4.965, 52.373)
@iot.id: 2215, name: LTD_23231, description: LTD_23231, location: (4.926, 52.358)
@iot.id: 720, name: LTD_8802, description: LTD_8802, location: (4.92, 52.402)
@iot.id: 2768, name: LTD_32999, description: LTD_32999, location: (4.89, 52.412)
@iot.id: 5964, name: LTD_55037, description: LTD_55037, location: (4.992, 52.354)


In [33]:
import requests
import csv

# List of LTD values
ltd_values = ['LTD_4969', 'LTD_32517', 'LTD_23231', 'LTD_8802', 'LTD_32999', 'LTD_55037']
headers = {"Content-Type": "application/json"}
output_path = "samenmeten_locations.csv"

def get_coordinates(thing):
    # Step 1: Extract coordinates from the location link
    locations_link = thing.get('Locations@iot.navigationLink')
    if not locations_link:
        return None, None

    location_response = requests.get(locations_link, headers=headers)
    if location_response.status_code != 200:
        return None, None

    location_data = location_response.json()
    coordinates = location_data.get('value', [{}])[0].get('location', {}).get('coordinates', [])
    return coordinates if len(coordinates) == 2 else (None, None)

# Step 2: Open a CSV file for writing
with open(output_path, "w", newline='') as csvfile:
    # Step 3: Define field names for CSV
    fieldnames = ['sensor_id', 'name', 'description', 'longitude', 'latitude']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    # Step 4: Write CSV header
    writer.writeheader()

    # Step 5: Loop through each LTD value
    for ltd_value in ltd_values:
        # Step 6: Make API request for each LTD value
        url = f"https://api-samenmeten.rivm.nl/v1.0/Things?$filter=name eq '{ltd_value}'"
        response = requests.get(url, headers=headers)

        # Step 7: Check response status
        if response.status_code == 200 and 'value' in (data := response.json()):
            # Step 8: Loop through sensor data
            for thing in data['value']:
                sensor_id, name, description = thing.get('@iot.id'), thing.get('name'), thing.get('description')
                lng, lat = get_coordinates(thing)
                
                # Step 9: Write sensor data to CSV
                writer.writerow({'sensor_id': sensor_id, 'name': name, 'description': description, 'longitude': lng, 'latitude': lat})
        else:
            # Step 10: Print error message if there's an issue with the API request
            print(f"Error for {ltd_value}: {response.status_code}")

# Step 11: Print a message indicating that data has been written to the CSV file
print(f"Data written to {output_path}")

Data written to samenmeten_locations.csv
